# 30 Day Map Challenge: Day 5 (Ukraine)

### Using Folium to Create a Map of Global Protests for Ukraine
Reina Chano Murray  |  November 5, 2022  |  [github/reinacmurray](https://github.com/reinacmurray)  
 

### My approach to the 30 Day Map Challenge
In my day job, I spend more time advising people on making map applications and/or admninistering GIS systems, so I decided to use the 2022 [30 Day Map Challenge](https://30daymapchallenge.com/) as an avenue to 1) play around with creating maps programmatically and 2) explore software and platforms I haven't had a chance to use before. And of course, just have fun making maps! 

### The background on this notebook
I used [leafmap](https://leafmap.org/) for days 1-4 of the 30 Day Map Challenge, but for Day 5, the theme is "Ukraine" -- I decided to explicitly use [folium](https://python-visualization.github.io/folium/), which is a python package for to visualize maps in leaflet. 

This map was created by scraping data from [USA Today's research on global protests in support of Ukraine](https://www.usatoday.com/in-depth/graphics/2022/03/05/russia-ukraine-protests-location-map/6981712001/). I scraped the data from the table at the end of the article, which contained just the city, country, and a link to a news article for that particular protest. 

Instead of using a geocoder, I used [Simple Map's World Cities database](https://simplemaps.com/data/world-cities), bringing it in as a csv and joining it to my protest data. Some manual editing needed to be done, particularly for the US, where there are far too many cities with the same name in different states. Because the USA Today article did not include other administrative information, like State names, I clicked through to the article link to confirm which city dupes to keep and which to remove. 

Once the editing was completed, a cleaned csv was brought back in and mapped using Folium. 

Using the code and information from this [article](https://towardsdatascience.com/the-battle-of-interactive-geographic-visualization-part-5-folium-cc2213d29a7), the Marker Cluster plugin was used to cluster points together. 

In [1]:
# Import necessary modules
import pandas as pd
import folium 
from folium.plugins import MarkerCluster

# Read the data
cities_protests = pd.read_csv("cities_protest.csv", sep=",")   # scraped from https://www.usatoday.com/in-depth/graphics/2022/03/05/russia-ukraine-protests-location-map/6981712001/
global_cities = pd.read_csv("worldcities.csv", sep=",")     # pulled from https://simplemaps.com/data/world-cities

In [2]:
# change "USA" to "United States"
# change "UK" to "United Kingdom"
cities_protests.loc[cities_protests["Country"] == "USA", "Country"] = "United States"
cities_protests.loc[cities_protests["Country"] == "UK", "Country"] = "United Kingdom"

In [3]:
# use a left join to join the information from global cities to protest cities
joined_table = pd.merge(cities_protests, global_cities, 
                 how="left", left_on = ["City", "Country"], right_on = ["city_ascii", "country"])

joined_table

,City,Country,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tirana,Albania,Tirana,Tirana,41.3300,19.8200,Albania,AL,ALB,Tiranë,primary,418495.0,1.008162e+09
1,Andorra la Vella,Andorra,Andorra la Vella,Andorra la Vella,42.5000,1.5000,Andorra,AD,AND,Andorra la Vella,primary,22615.0,1.020829e+09
2,Buenos Aires,Argentina,Buenos Aires,Buenos Aires,-34.5997,-58.3819,Argentina,AR,ARG,"Buenos Aires, Ciudad Autónoma de",primary,16216000.0,1.032717e+09
3,Yerevan,Armenia,Yerevan,Yerevan,40.1814,44.5144,Armenia,AM,ARM,Yerevan,primary,1075800.0,1.051074e+09
4,Sydney,Australia,Sydney,Sydney,-33.8650,151.2094,Australia,AU,AUS,New South Wales,admin,4840600.0,1.036075e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,Concord,United States,Concord,Concord,42.5450,-78.7075,United States,US,USA,New York,NaN,8484.0,1.840058e+09
291,Ames,United States,Ames,Ames,42.0259,-93.6215,United States,US,USA,Iowa,NaN,67910.0,1.840007e+09
292,Kherson,Ukraine,Kherson,Kherson,46.6333,32.6000,Ukraine,UA,UKR,Khersons’ka Oblast’,admin,291428.0,1.804514e+09
293,Mariupol,Ukraine,Mariupol,Mariupol,47.1306,37.5639,Ukraine,UA,UKR,Donets’ka Oblast’,NaN,449498.0,1.804630e+09


In [4]:
# check for any where a join was not successful

nan_in_col  = joined_table[joined_table["city"].isna()]
len(nan_in_col)

11

In [5]:
# view unsuccessful joins
nan_in_col

,City,Country,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
13,Sarajevo,Bosnia and Herzegovina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,Hong Kong,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,Prague,Czech Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,Brno,Czech Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,Tel Aviv,Israel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,Luxembourg City,Luxembourg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,Kappara,Malta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,Skopje,North Madeonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,St. Petersburg,Russia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,Nizhny Novgorod,Russia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# check US -- lots of dupes -_-
joined_table.loc[joined_table["Country"] == "United States"]

,City,Country,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
138,New York,United States,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,18713220.0,1.840034e+09
139,"Washington, D.C.",United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,Detroit,United States,Detroit,Detroit,42.3834,-83.1024,United States,US,USA,Michigan,NaN,3506126.0,1.840004e+09
141,San Francisco,United States,San Francisco,San Francisco,37.7562,-122.4430,United States,US,USA,California,NaN,3592294.0,1.840022e+09
142,Montclair,United States,Montclair,Montclair,34.0714,-117.6980,United States,US,USA,California,NaN,40083.0,1.840020e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,Concord,United States,Concord,Concord,42.4620,-71.3639,United States,US,USA,Massachusetts,NaN,19116.0,1.840054e+09
288,Concord,United States,Concord,Concord,38.5117,-90.3574,United States,US,USA,Missouri,NaN,17912.0,1.840006e+09
289,Concord,United States,Concord,Concord,39.8741,-75.5135,United States,US,USA,Pennsylvania,NaN,17745.0,1.840153e+09
290,Concord,United States,Concord,Concord,42.5450,-78.7075,United States,US,USA,New York,NaN,8484.0,1.840058e+09


In [7]:
# save to a csv for manual editing
joined_table.to_csv("geocoded_cities_protest_raw.csv", sep=",", encoding="utf-8", index=False)


In [8]:
# re-upload geocoded csv after manual edits complete
geocoded_cities = pd.read_csv("geocoded_cities_protest.csv", sep=",")


In [9]:
# keep key columns
geocoded_cities = geocoded_cities[["City", "Country", "lat", "lng", "population", "id"]]
geocoded_cities

,City,Country,lat,lng,population,id
0,Tirana,Albania,41.3300,19.8200,418495.0,1008162156
1,Andorra la Vella,Andorra,42.5000,1.5000,22615.0,1020828846
2,Buenos Aires,Argentina,-34.5997,-58.3819,16216000.0,1032717330
3,Yerevan,Armenia,40.1814,44.5144,1075800.0,1051074169
4,Sydney,Australia,-33.8650,151.2094,4840600.0,1036074917
...,...,...,...,...,...,...
197,Concord,United States,43.2305,-71.5595,43627.0,1840002747
198,Ames,United States,42.0259,-93.6215,67910.0,1840007019
199,Kherson,Ukraine,46.6333,32.6000,291428.0,1804514036
200,Mariupol,Ukraine,47.1306,37.5639,449498.0,1804630021


In [21]:
# make a map with folium!
# center the map on Kyiv, Ukraine ([50.45, 30.54])
ukraine_protests_map = folium.Map(location=[50.45, 30.54]
                 , zoom_start=2, 
                 control_scale=True)

	
folium.TileLayer('cartodbpositron').add_to(ukraine_protests_map)


In [22]:
# create a marker cluster 
marker_cluster = MarkerCluster(control=False).add_to(ukraine_protests_map)


In [23]:
#We need this cause apparently, folium can recognize not a geoseries but a normal tuple/list
geocoded_cities.reset_index(drop=True, inplace=True)
geocoded_cities['geocode'] = [[geocoded_cities['lat'][i],geocoded_cities['lng'][i]] for i in range(len(geocoded_cities)) ]



In [24]:
for point in geocoded_cities.index: 
    # loop through the plots
    html = """<b>{city}, {country}</b><br><p>population: {pop}</p>"""
    popup_contents = folium.Html(html.format(city = geocoded_cities.loc[point, 'City'], 
                                            country = geocoded_cities.loc[point, 'Country'],
                                            pop = geocoded_cities.loc[point, 'population']),
                                 script = True)
    
    popup = folium.Popup(popup_contents, max_width=1500)
    
    folium.Marker(geocoded_cities['geocode'][point],
                     popup=popup).add_to(marker_cluster)

In [28]:
ukraine_protests_map